In [7]:
import tensorflow as tf

In [10]:
SAVED_MODEL_DIR = './savedmodeldir/'

eff_det = tf.saved_model.load(SAVED_MODEL_DIR)

In [16]:
eff_det.__class__

tensorflow.python.trackable.autotrackable.AutoTrackable